In [ ]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import model_afolu as mafl
import model_circular_economy as mc
import model_ippu as mi
import model_socioeconomic as ms

importlib.reload(ma)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(mafl)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(ms)

In [ ]:
df_tmp = pd.read_csv("/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/ingestion/FAOSTAT/Forestry_E_All_Data_(Normalized)/Forestry_E_All_Data_(Normalized)_utf8.csv")

yrs = range(2011, 2020)

dict_add = {"Wood fuel": 0.5, "Industrial roundwood": 1, "Pulpwood, round and split, all species (production)": -1}
items = list(dict_add.keys())

df_tmp2 = df_tmp[
    #df_tmp["Area"].isin(["Mexico"]) &
    df_tmp["Year"].isin(yrs) & 
    df_tmp["Unit"].isin(["m3"]) &
    df_tmp["Element"].isin(["Production"]) & 
    df_tmp["Item"].isin(items)
].copy()
df_tmp2["Item_weight"] = df_tmp2["Item"].replace(dict_add)
df_tmp2["sum_val"] = np.array(df_tmp2["Item_weight"])*np.array(df_tmp2["Value"])

scalar_m3_to_tonnes = 0.5

fields_group = ["Area", "Year"]
fields_agg = ["sum_val"]
dict_agg = dict(zip(fields_group, ["first" for x in fields_group]))
dict_agg.update(dict(zip(fields_agg, ["sum" for x in fields_agg])))
df_tmp3 = df_tmp2.groupby(fields_group).agg(dict_agg).reset_index(drop = True)
field_new = "prodinit_ippu_wood_tonne"
df_tmp3.rename(columns = {"sum_val": field_new}, inplace = True)
df_tmp3[field_new] = np.array(df_tmp3[field_new])*scalar_m3_to_tonnes


df_persons = df_tmp[
    #df_tmp["Area"].isin(["Mexico"]) &
    df_tmp["Year"].isin(yrs) & 
    df_tmp["Unit"].isin(["m3"]) &
    df_tmp["Element"].isin(["Production"]) & 
    df_tmp["Item"].isin(["Wood fuel"])
].copy()



field_new_2 = "deminit_frst_wood_home_use_kg_per_hh_NOTEDIVIDEBYPOPULATIONFIRST"
df_persons[field_new_2] = np.array(df_persons["Value"]*(1 - dict_add["Wood fuel"]))*scalar_m3_to_tonnes*1000


df_out = pd.merge(df_tmp3, df_persons[fields_group + [field_new_2]])
           
#df_tmp3[field_new] = np.array(df_tmp3[field_new])*0.5

#].to_csv("/Users/jsyme/Desktop/tmp.csv")